In [ ]:
!!pip install litellm


In [1]:
# For local VSCODE and Jupyter ONLY:

import os
from litellm import completion
from dotenv import load_dotenv
from typing import List, Dict


load_dotenv()

HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# print(HUGGINGFACE_API_KEY)
# print(OLLAMA_API_KEY)
# print(GEMINI_API_KEY)

API_KEY = GEMINI_API_KEY # HUGGINGFACE_API_KEY OR OLLAMA_API_KEY # OR GEMINI_API_KEY

In [2]:
from litellm import completion
from typing import List, Dict
import json


def generate_response(messages: List[Dict]) -> str:
    """Call LLM to get response"""
    response = completion(
        model="gemini/gemini-2.0-flash", 
        messages=messages,
        max_tokens=1024,
        api_key=API_KEY
    )
    return response.choices[0].message.content


In [4]:
def extract_markdown_block(response: str) -> str:
   """Extract code block from response"""

   if not '```' in response:
      return response

   code_block = response.split('```')[1].strip()
   # Check for "python" at the start and remove

#    if code_block.startswith("python"):
#       code_block = code_block[6:]

   return code_block

def parse_action(response: str) -> Dict:
        """Parse the LLM response into a structured action dictionary."""
        try:
            response = extract_markdown_block(response, "action")
            response_json = json.loads(response)
            if "tool_name" in response_json and "args" in response_json:
                return response_json
            else:
                return {"tool_name": "error", "args": {"message": "You must respond with a JSON tool invocation."}}
        except json.JSONDecodeError:
            return {"tool_name": "error", "args": {"message": "Invalid JSON response. You must respond with a JSON tool invocation."}}

In [6]:

max_iterations = 5
iterations = 0

agent_rules = [{
    "role": "system",
    "content": """
You are an AI agent that can perform tasks by using available tools.

Available tools:
- list_files() -> List[str]: List all files in the current directory.
- read_file(file_name: str) -> str: Read the content of a file.
- terminate(message: str): End the agent loop and print a summary to the user.

If a user asks about files, list them before reading.

Every response MUST have an action.
Respond in this format:

```action
{
    "tool_name": "insert tool_name",
    "args": {...fill in any required arguments here...}
}
```    
"""
}]
memory = []


while iterations < max_iterations:

    # 1. Construct prompt: Combine agent rules with memory
    prompt = agent_rules + memory

    # 2. Generate response from LLM
    #print("Agent thinking...")
    response = generate_response(prompt)
    print(f"Agent response no {iterations} : {response}")

    # 3. Parse response to determine action
    action = parse_action(response)

    result = ""

    if action["tool_name"] == "list_files":
        result = {"result: ": os.listdir('.//')}
    elif action["tool_name"] == "read_file":
        result = {"result: ":read_file(action["args"]["file_name"])}
    elif action["tool_name"] == "error":
        result = {"error: ":action["args"]["message"]}
    elif action["tool_name"] == "terminate":
        print(action["args"]["message"])
        break
    else:
        result = {"error":"Unknown action: "+action["tool_name"]}

    print(f"Action result: {result}")

    # 5. Update memory with response and results
    memory.append (
        {"role": "assistant", "content": response},
        {"role": "user", "content": json.dumps(result)}
    )

    # 6. Check termination condition
    if action["tool_name"] == "terminate":
        break

    iterations += 1

    


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



BadRequestError: litellm.BadRequestError: VertexAIException BadRequestError - {
  "error": {
    "code": 400,
    "message": "* GenerateContentRequest.contents: contents is not specified\n",
    "status": "INVALID_ARGUMENT"
  }
}
